In [ ]:
import pandas as pd

chunk_size = 500_000
result = []

group_cols = [
    "ta_ymd",
    "admi_cty_no",
    "card_tpbuz_nm_2",
    "hour",
    "sex",
    "age",
    "day"
]

age_map = {
    3: 2,   # 20대
    4: 3,   # 30대
    5: 4,   # 40대
    6: 5,   # 50대
    7: 6,   # 60대 이상
    8: 6,
    9: 6,
    10: 6,
    11: 6
}

category_map = {
    "닭/오리요리": "고기요리",
    "커피/음료": "디저트/음료",
    "제과/제빵/떡/케익": "디저트/음료"
}

for chunk in pd.read_csv(
    "../food/202201.csv",
    chunksize=chunk_size
):
    # 1️⃣ 10대 이하 제거
    chunk = chunk[chunk["age"] >= 3]

    # 2️⃣ age 재매핑
    chunk["age"] = chunk["age"].map(age_map)
    chunk = chunk.dropna(subset=["age"])
    chunk["age"] = chunk["age"].astype(int)

    # 3️⃣ 업종 통합 (🔥 핵심 추가)
    chunk["card_tpbuz_nm_2"] = chunk["card_tpbuz_nm_2"].replace(category_map)

    # 4️⃣ 집계
    chunk_agg = (
        chunk
        .groupby(group_cols, as_index=False)
        .agg({"amt": "sum", "cnt": "sum"})
    )

    result.append(chunk_agg)
    
df_final = pd.concat(result, ignore_index=True)

df_final = (
    df_final
    .groupby(group_cols, as_index=False)
    .agg({
        "amt": "sum",
        "cnt": "sum"
    })
)

In [10]:
df_final = pd.concat(result, ignore_index=True)

df_final = (
    df_final
    .groupby(group_cols, as_index=False)
    .agg({
        "amt": "sum",
        "cnt": "sum"
    })
)

In [11]:
df_final["card_tpbuz_nm_2"].value_counts()


card_tpbuz_nm_2
디저트/음료     610326
한식         534275
고기요리       412418
분식         399493
패스트푸드      381487
일식/수산물     283315
별식/퓨전요리    241505
중식         239609
양식         144218
간이주점       138700
Name: count, dtype: int64

In [12]:
print(len(df_final))

3385346


=============================================================

In [13]:
import os
import pandas as pd

def process_and_overwrite(file_path):
    temp_path = file_path.replace(".csv", "_tmp.csv")

    chunk_size = 500_000

    group_cols = [
        "ta_ymd",
        "admi_cty_no",
        "card_tpbuz_nm_2",
        "hour",
        "sex",
        "age",
        "day"
    ]

    age_map = {
        3: 2, 4: 3, 5: 4, 6: 5,
        7: 6, 8: 6, 9: 6, 10: 6, 11: 6
    }

    category_map = {
        "닭/오리요리": "고기요리",
        "커피/음료": "디저트/음료",
        "제과/제빵/떡/케익": "디저트/음료"
    }

    first_write = True

    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        # 1️⃣ 10대 이하 제거
        chunk = chunk[chunk["age"] >= 3]

        # 2️⃣ age 재매핑
        chunk["age"] = chunk["age"].map(age_map)
        chunk = chunk.dropna(subset=["age"])
        chunk["age"] = chunk["age"].astype(int)

        # 3️⃣ 업종 통합
        chunk["card_tpbuz_nm_2"] = chunk["card_tpbuz_nm_2"].replace(category_map)

        # 4️⃣ chunk 단위 집계
        chunk_agg = (
            chunk
            .groupby(group_cols, as_index=False)
            .agg({"amt": "sum", "cnt": "sum"})
        )

        # 5️⃣ 바로 파일에 기록 (append)
        chunk_agg.to_csv(
            temp_path,
            mode="w" if first_write else "a",
            header=first_write,
            index=False
        )
        first_write = False

    # 🔥 tmp → 원본 파일로 덮어쓰기
    os.replace(temp_path, file_path)


In [14]:
data_dir = "../food"

files = [
    f"{year}{month:02d}.csv"
    for year in range(2022, 2026)
    for month in range(1, 13)
    if not (year == 2025 and month > 11)
]

for f in files:
    file_path = os.path.join(data_dir, f)

    if not os.path.exists(file_path):
        continue

    print(f"▶ 처리 중: {f}")
    process_and_overwrite(file_path)
    print(f"✅ 덮어쓰기 완료: {f}")


▶ 처리 중: 202201.csv
✅ 덮어쓰기 완료: 202201.csv
▶ 처리 중: 202202.csv
✅ 덮어쓰기 완료: 202202.csv
▶ 처리 중: 202203.csv
✅ 덮어쓰기 완료: 202203.csv
▶ 처리 중: 202204.csv
✅ 덮어쓰기 완료: 202204.csv
▶ 처리 중: 202205.csv
✅ 덮어쓰기 완료: 202205.csv
▶ 처리 중: 202206.csv
✅ 덮어쓰기 완료: 202206.csv
▶ 처리 중: 202207.csv
✅ 덮어쓰기 완료: 202207.csv
▶ 처리 중: 202208.csv
✅ 덮어쓰기 완료: 202208.csv
▶ 처리 중: 202209.csv
✅ 덮어쓰기 완료: 202209.csv
▶ 처리 중: 202210.csv
✅ 덮어쓰기 완료: 202210.csv
▶ 처리 중: 202211.csv
✅ 덮어쓰기 완료: 202211.csv
▶ 처리 중: 202212.csv
✅ 덮어쓰기 완료: 202212.csv
▶ 처리 중: 202301.csv
✅ 덮어쓰기 완료: 202301.csv
▶ 처리 중: 202302.csv
✅ 덮어쓰기 완료: 202302.csv
▶ 처리 중: 202303.csv
✅ 덮어쓰기 완료: 202303.csv
▶ 처리 중: 202304.csv
✅ 덮어쓰기 완료: 202304.csv
▶ 처리 중: 202305.csv
✅ 덮어쓰기 완료: 202305.csv
▶ 처리 중: 202306.csv
✅ 덮어쓰기 완료: 202306.csv
▶ 처리 중: 202307.csv
✅ 덮어쓰기 완료: 202307.csv
▶ 처리 중: 202308.csv
✅ 덮어쓰기 완료: 202308.csv
▶ 처리 중: 202309.csv
✅ 덮어쓰기 완료: 202309.csv
▶ 처리 중: 202310.csv
✅ 덮어쓰기 완료: 202310.csv
▶ 처리 중: 202311.csv
✅ 덮어쓰기 완료: 202311.csv
▶ 처리 중: 202312.csv
✅ 덮어쓰기 완료: 202312.csv
▶ 처리 중: 202401.c

In [15]:
df = pd.read_csv(file_path)
df = (
    df
    .groupby(group_cols, as_index=False)
    .agg({"amt": "sum", "cnt": "sum"})
)
df.to_csv(file_path, index=False)
